Bu örneğimizde, RSA şifreleme işlemi gerçekleştiren XMEGA hedef kartından Basit Güç Analizi ile anahtarı elde etmeye çalışacağız.

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW303'
CRYPTO_TARGET = 'AVRCRYPTOLIB'

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Simpleserial içerisinden rsa için oluşturulan yazılımı hedef kartımıza yüklüyoruz.

In [ ]:
fw_path = '../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-CW303.hex'
cw.program_target(scope, prog, fw_path)

Sample sayımızı 24000 olarak ayarlıyoruz. Uzun bir grafik elde edip işleme geniş açıdan bakacağız. 

In [ ]:
scope.adc.samples = 24000
scope.clock.adc_src = "clkgen_x1"
#RSA is slow – we capture at 1x speed & also extend capture size.
#clkgen_x1”: CLKGEN output via DCM

#“clkgen_x4”: CLKGEN output via DCM with x4 clk multiplier

#“extclk_x1”: External clock input via DCM

#“extclk_x4”: External clock input via DCM with x4 clk multiplier

#“extclk_dir”: External clock input with no DCM

Başlangıçta giriş ve anahtar için boş değerler atıyoruz.  
Hedef kodumuz içerisinde anahtar barındırmadığı için dışarıdan biz vereceğiz anahtar.

In [ ]:
target.init_cmd = ""
target.input_cmd = ""
target.key_cmd = ""

Anahtara belirli değerler vererek RSA şifrelemesinin sonucunun grafiğini inceliyoruz. (Anahtarımız son tanımladığımız oluyor.)

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
key = bytearray ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0x80, 0x00])
trace = cw.capture_trace (scope, target, key,key)
#plt.plot(trace.wave ,'r')
key = bytearray ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0x80,0x10])
trace = cw.capture_trace (scope, target, key, key)
#plt.plot(trace.wave, 'b')
key = bytearray ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0x8A,0xA0])
trace = cw.capture_trace (scope, target, key, key)
plt.plot(trace.wave, 'k')

Belirli bir noktaya kadar eş giden grafikte sonrasında gecikmeler ve bozulmalar görülmektedir. Burada değişime başlanmadan önceki bir noktayı referans numarası olarak kabul edip başlangıçtan itibaren tüm değerlerin farkını alarak ilk değerden çıkarıyoruz. Devam eden işlemin sinyal değişiklikleri hakkında bilgi edinmiş oluyoruz ve bununla güç izimiz anahtarı anlayabileceğimiz forma geliyor.

In [ ]:
%matplotlib notebook
import numpy as np
start = 3600
rsa_one = trace.wave[start:(start+500)]
diffs = []
for i in range (0, 23499):
    diff = trace.wave[i:(i+len(rsa_one))] - rsa_one
    diffs.append(np.sum(abs(diff)))
plt.figure()
plt.plot(diffs)

Farkın az olduğu noktaları işlem farklılığı hakkında bilgi verdiği için seçiyoruz. Bu grafiğe göre 10'dan düşük değere sahip noktalar bize işlem hakkında bilgi verecektir. Hangi noktalarda olduğuna bakıyoruz.

In [ ]:
import numpy as np
np.where(np.array(diffs)<10)

Bulduğumuz nokta aralıkları arasındaki farkı belirli bir referans değeri ile karşılaştırıp anahtarımızı yazıyoruz.
*Burada belirlediğimiz referans değeri 10'dan küçük değerlerin olduğu indislere göre belirlenir.

In [ ]:
import numpy as np
key = ""
times = np.where(np.array(diffs)<10)[0]
for i in range (0, len(times)-1):
    delta = times[i+1] - times[i]
    #print (delta)
    if delta > 1500:
        key += "1"
    else:
        key += "0"
key += "0"
print(key)
print("%04X"%int(key,2))

Grafiği daha iyi görebilmek için filtre de uygulayabiliriz. Burada alçak geçiren filtre uygulayarak sinyali daha anlaşılır ve gürültülerden arınmış şekilde görebiliriz.

In [ ]:
%matplotlib notebook
from scipy.signal import butter , lfilter

b, a = butter(3, 0.001, btype="low")
y = lfilter(b, a, trace.wave)

plt.plot(y)

In [ ]:
scope.dis()
target.dis()
#bu kod chipwhisperer aracımız ile olan bağlantıyı sonlandırır.